In [1]:
import numpy as np
import pandas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
df=pd.read_csv("file-path")

In [3]:
df.head()

,bike_name,price,city,kms_driven,owner,age,power,brand,Original Price
0,TVS Star City Plus Dual Tone 110cc,35000,Ahmedabad,17654,First Owner,3,110,TVS,82500
1,Royal Enfield Classic 350cc,119900,Delhi,11000,First Owner,4,350,Royal Enfield,230000
2,Triumph Daytona 675R,600000,Delhi,110,First Owner,8,675,Triumph,1100000
3,TVS Apache RTR 180cc,65000,Bangalore,16329,First Owner,4,180,TVS,140000
4,Yamaha FZ S V 2.0 150cc-Ltd. Edition,80000,Bangalore,10000,First Owner,3,150,Yamaha,150000


In [10]:

for col in ["price",'Original Price']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower) & (df[col] <= upper)]

In [12]:
column=['bike_name','city']
df.drop(column,inplace=True,axis=1)

In [13]:
df['year']=2025-df['age']
df.drop('age',errors='ignore',axis=1,inplace=True)

In [14]:
df.columns

Index(['price', 'kms_driven', 'owner', 'power', 'brand', 'Original Price',
       'year'],
      dtype='object')

In [ ]:
x=df[['kms_driven', 'owner', 'power', 'brand',
       'Original Price', 'year']]
y=df['price']
# train test split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
# preprocessing: numeric and categorical
num_feats=x_train.select_dtypes(include=[np.number]).columns.tolist()
cat_feats=x_train.select_dtypes(include=['object','category']).columns.tolist()

num_transformer=Pipeline([
    ('scaler',StandardScaler())
])

cat_transformer=Pipeline([
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor=ColumnTransformer([
    ('num',num_transformer,num_feats),
    ('cat',cat_transformer,cat_feats)
])
# full pipeline:preprocessing + model
model=Pipeline([
    ('preproc',preprocessor),
    ('regressor',LinearRegression())
])
# fit the model
model.fit(x_train,y_train)

,steps,"[('preproc', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [17]:
y_train_pred=model.predict(x_train)
y_test_pred = model.predict(x_test)

def rmse(a,b):return np.sqrt(mean_squared_error(a,b))

print("Train R²:", r2_score(y_train, y_train_pred))
print("Train RMSE:", rmse(y_train, y_train_pred))

print("Test R²:", r2_score(y_test, y_test_pred))
print("Test RMSE:", rmse(y_test, y_test_pred))

Train R²: 0.8308732685746136
Train RMSE: 17494.11497043122
Test R²: 0.8338514573055182
Test RMSE: 18042.25344696056


In [ ]:
# Alternative: using pickle
import pickle

# Save
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

# Load
with open("model.pkl", "rb") as f:
    loaded_model = pickle.load(f)


In [23]:
from statsmodels.api import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
x=add_constant(df1)
vif=[variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
for features, i in zip(x.columns[1:],vif[1:]):
    print(features ,":",i)
# if vif ==1 no multicollinearity
# if vif>5 has multicollinearity so drop the column

kms_driven : 1.14521173661188
power : 4.93168724912342
Original Price : 5.088932866887556
year : 1.1896130791634756


In [24]:
y=df['price']
import statsmodels.api as sm
model = sm.OLS(y,x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     7044.
Date:                Mon, 08 Dec 2025   Prob (F-statistic):               0.00
Time:                        18:00:46   Log-Likelihood:                -78431.
No. Observations:                6956   AIC:                         1.569e+05
Df Residuals:                    6951   BIC:                         1.569e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -5.019e+06   1.29e+05    -38.